Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [9]:
pickle_file = 'notMNIST.pickle'
sample_size = 5000 #prevent memory error on g2.2xlarge
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset'][:sample_size]
  valid_labels = save['valid_labels'][:sample_size]
  test_dataset = save['test_dataset'][:sample_size]
  test_labels = save['test_labels'][:sample_size]
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [10]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (5000, 28, 28, 1) (5000, 10)
Test set (5000, 28, 28, 1) (5000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(0.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.670947
Minibatch accuracy: 12.5%
Validation accuracy: 12.0%
Minibatch loss at step 50: 2.021527
Minibatch accuracy: 50.0%
Validation accuracy: 62.7%
Minibatch loss at step 100: 0.970926
Minibatch accuracy: 68.8%
Validation accuracy: 73.2%
Minibatch loss at step 150: 0.396535
Minibatch accuracy: 93.8%
Validation accuracy: 79.0%
Minibatch loss at step 200: 0.399272
Minibatch accuracy: 87.5%
Validation accuracy: 79.1%
Minibatch loss at step 250: 1.253738
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 300: 0.318013
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 350: 0.746748
Minibatch accuracy: 75.0%
Validation accuracy: 80.7%
Minibatch loss at step 400: 0.723352
Minibatch accuracy: 75.0%
Validation accuracy: 82.5%
Minibatch loss at step 450: 0.614430
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
Minibatch loss at step 500: 0.238736
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [17]:
batch_size = 128
image_size = 28
num_channels = 1
depth = 16
patch_size = 5
num_labels = 10
pool_stride = 2
pool_kernel_size = 2
fc_hidden_nodes = 1024


graph = tf.Graph()
with graph.as_default():
    tf_batch_data = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_batch_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_data = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)
    
    conv_W1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    conv_b1 = tf.Variable(tf.zeros([depth]))
    conv_W2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    conv_b2 = tf.Variable(tf.zeros([depth]))
    
    fc_size = (image_size//pool_stride**2)**2 * depth
    fc_W1 = tf.Variable(tf.truncated_normal([fc_size, fc_hidden_nodes], stddev=0.1))
    fc_b1 = tf.Variable(tf.zeros([fc_hidden_nodes]))
    fc_W2 = tf.Variable(tf.truncated_normal([fc_hidden_nodes, num_labels], stddev=0.1))
    fc_b2 = tf.Variable(tf.zeros([num_labels]))
    
    def model(data):
        conv = tf.nn.conv2d(data, conv_W1, [1,1,1,1], padding='SAME')
        hidden = tf.nn.relu(conv + conv_b1)
        
        hidden = tf.nn.max_pool(hidden, [1, pool_kernel_size, pool_kernel_size, 1], [1, pool_stride, pool_stride, 1], padding='VALID')
        print(hidden.get_shape().as_list())
        conv = tf.nn.conv2d(hidden, conv_W2, [1,1,1,1], padding='SAME')
        hidden = tf.nn.relu(conv + conv_b1)
        
        hidden = tf.nn.max_pool(hidden, [1, pool_kernel_size, pool_kernel_size, 1], [1, pool_stride, pool_stride, 1], padding='VALID')
        
        shape = hidden.get_shape().as_list()
        hidden = tf.reshape(hidden, [shape[0], shape[1]*shape[2]*shape[3]] )
        hidden = tf.matmul(hidden, fc_W1)
        hidden = hidden + fc_b1
        hidden = tf.nn.relu(hidden)
        print(hidden.get_shape().as_list())
        return tf.matmul(hidden, fc_W2) + fc_b2

    logits = model(tf_batch_data)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_batch_labels))

    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_data))
    test_prediction = tf.nn.softmax(model(tf_test_data))
    

[128, 14, 14, 16]
[128, 1024]
[10000, 14, 14, 16]
[10000, 1024]
[10000, 14, 14, 16]
[10000, 1024]


In [8]:
num_steps = 1001

with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = batch_size * step % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {
            tf_batch_data: batch_data,
            tf_batch_labels: batch_labels
        }
        op, l, p = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step % 100 == 0:
            print("step: %5d, loss:%.2f" % (step, l))
            print("train accuracy:%.2f%%" % accuracy(p, batch_labels))
            print("valid accuracy:%.2f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("test accuracy:%.2f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
step:     0, loss:2.76
train accuracy:7.03%
valid accuracy:16.10%
step:   100, loss:0.67
train accuracy:80.47%
valid accuracy:81.50%
step:   200, loss:0.80
train accuracy:77.34%
valid accuracy:82.40%
step:   300, loss:0.60
train accuracy:84.38%
valid accuracy:84.12%
step:   400, loss:0.70
train accuracy:77.34%
valid accuracy:85.40%
step:   500, loss:0.56
train accuracy:85.16%
valid accuracy:86.14%
step:   600, loss:0.50
train accuracy:85.16%
valid accuracy:86.66%
step:   700, loss:0.51
train accuracy:85.94%
valid accuracy:86.90%
step:   800, loss:0.58
train accuracy:84.38%
valid accuracy:87.27%
step:   900, loss:0.27
train accuracy:93.75%
valid accuracy:87.96%
step:  1000, loss:0.41
train accuracy:89.84%
valid accuracy:87.94%
test accuracy:93.64%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [11]:
batch_size = 128
image_size = 28
num_channels = 1
depth = 16
depth2 = 16
patch_size = 5
num_labels = 10
pool_stride = 2
pool_ksize = 2
fc1_hidden_hodes = 1024
fc1_W_size = (image_size//pool_stride**2)**2 * depth2
decay_steps = 1000
decay_rate = 0.96

graph = tf.Graph()
with graph.as_default():
    
  tf_batch_data = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_batch_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_data = tf.constant(valid_dataset)
  tf_test_data = tf.constant(test_dataset)
  keep_rate = tf.placeholder(tf.float32)
  global_step = tf.Variable(0)
  

  conv_filter1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  conv_bias1 = tf.Variable(tf.zeros([depth]))
  conv_filter2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.01))
  conv_bias2 = tf.Variable(tf.zeros([depth]))

  #max pool 1 stride 2, 2, size 2, 2

  conv_filter3 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth2], stddev=0.1))
  conv_bias3 = tf.Variable(tf.zeros([depth2]))    
  conv_filter4 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth2, depth2], stddev=0.01))
  conv_bias4 = tf.Variable(tf.zeros([depth2]))

  #max pool 2 stride 2, 2, size 2, 2
  #reshape

  #fully connected layer
  fc1_W = tf.Variable(tf.truncated_normal([fc1_W_size, fc1_hidden_hodes], stddev=0.1))
  fc1_b = tf.Variable(tf.zeros([fc1_hidden_hodes]))

  fc2_W = tf.Variable(tf.truncated_normal([fc1_hidden_hodes, num_labels], stddev=0.1))
  fc2_b = tf.Variable(tf.zeros([num_labels]))

  def model(data):
    conv = tf.nn.conv2d(data, conv_filter1, [1,1,1,1], padding='SAME') + conv_bias1
    conv = tf.nn.relu(conv)
    conv = tf.nn.conv2d(conv, conv_filter2, [1,1,1,1], padding='SAME') + conv_bias2
    conv = tf.nn.relu(conv)
    conv = tf.nn.max_pool(conv, [1, pool_ksize, pool_ksize, 1], [1, pool_stride, pool_stride, 1], padding='VALID')
    print(conv.get_shape().as_list())
    conv = tf.nn.conv2d(conv, conv_filter3, [1,1,1,1], padding='SAME') + conv_bias3
    conv = tf.nn.relu(conv)
    conv = tf.nn.conv2d(conv, conv_filter4, [1,1,1,1], padding='SAME') + conv_bias4
    conv = tf.nn.relu(conv)

    
    conv = tf.nn.max_pool(conv, [1, pool_ksize, pool_ksize, 1], [1, pool_stride, pool_stride, 1], padding='VALID')
    shape = conv.get_shape().as_list()
    conv = tf.reshape(conv, [shape[0], shape[1]*shape[2]*shape[3]])
    conv = tf.nn.relu(tf.matmul(conv, fc1_W) + fc1_b)
    conv = tf.nn.dropout(conv, keep_rate)
    return tf.matmul(conv, fc2_W) + fc2_b

  logits = model(tf_batch_data)
    
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_batch_labels))
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss, global_step=global_step)

  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_data))
  test_prediction = tf.nn.softmax(model(tf_test_data))


[128, 14, 14, 16]
[5000, 14, 14, 16]
[5000, 14, 14, 16]


In [12]:
num_steps = 10001

with tf.Session(graph=graph) as sess:
  tf.initialize_all_variables().run()
  print('Initialized')

  for step in xrange(num_steps):
    offset = step * batch_size % (train_dataset.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset+batch_size),:,:,:]
    batch_labels = train_labels[offset:(offset+batch_size),:]
    feed_dict = {
      tf_batch_data: batch_data,
      tf_batch_labels: batch_labels,
      keep_rate: 0.5
    }

    op, l, p = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

    if step % 100 == 0:
      print("step: %5d, loss: %.2f" % (step, l))
      print("train: %.2f%%" % (accuracy(p, batch_labels)))
      print("valid: %.2f%%" % (accuracy(valid_prediction.eval(feed_dict={keep_rate:1.0}), valid_labels)))
  print("test: %.2f%%" % (accuracy(test_prediction.eval(feed_dict={keep_rate:1.0}), test_labels)))


Initialized
step:     0, loss: 2.30
train: 8.59%
valid: 10.00%
step:   100, loss: 0.82
train: 78.91%
valid: 79.80%
step:   200, loss: 0.88
train: 75.00%
valid: 82.42%
step:   300, loss: 0.62
train: 85.16%
valid: 83.58%
step:   400, loss: 0.80
train: 78.91%
valid: 85.06%
step:   500, loss: 0.67
train: 81.25%
valid: 86.10%
step:   600, loss: 0.54
train: 82.81%
valid: 86.12%
step:   700, loss: 0.52
train: 85.16%
valid: 86.20%
step:   800, loss: 0.68
train: 82.03%
valid: 87.16%
step:   900, loss: 0.34
train: 90.62%
valid: 87.08%
step:  1000, loss: 0.53
train: 87.50%
valid: 86.92%
step:  1100, loss: 0.43
train: 89.84%
valid: 87.52%
step:  1200, loss: 0.34
train: 89.06%
valid: 87.60%
step:  1300, loss: 0.40
train: 85.94%
valid: 88.40%
step:  1400, loss: 0.52
train: 85.16%
valid: 88.22%
step:  1500, loss: 0.42
train: 90.62%
valid: 88.06%
step:  1600, loss: 0.54
train: 82.81%
valid: 88.48%
step:  1700, loss: 0.43
train: 85.94%
valid: 88.70%
step:  1800, loss: 0.36
train: 91.41%
valid: 89.14%
s